In [1]:
import re
import os

In [18]:
test_texts_root = r'/home/nst/mount/data/share/yd/'\
                   'popular_science_texts_store/ner_markup/test_ner/test_texts/'
    
output_root = r'/home/nst/mount/data/share/yd/'\
               'popular_science_texts_store/ner_markup/test_ner/output_texts/'

In [111]:
file_name = 'nplus1.ru-news-2015-05-10-darkenergy.txt'
sample_path_in = test_texts_root + file_name
sample_path_out = output_root + file_name

In [3]:
def slurp(path):
    with open(path, 'r') as file_object:
        return file_object.read()
    
def spit(path, text):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, 'w') as file_object:
        return file_object.write(text)

In [47]:
text = slurp(sample_path_in)

In [53]:
spit(sample_path_out, text)

5021

In [19]:
def assertEqual(a, b):
    if a != b:
        raise AssertionError('expected "%s", actual "%s"' % (a, b))

In [31]:
def insert_tags(text, index_start, index_end):
    initial_tag = '<>'
    final_tag = '</>'
    return text[:index_start] + initial_tag + text[index_start:index_end] \
        + final_tag + text[index_end:]

assertEqual('ученый <>Хокинг</>', 
           insert_tags('ученый Хокинг', 7, 13))

In [32]:
def tag_text(text):
    pattern = re.compile(r'([А-ЯЁ][а-яё]+|[A-Z][a-z]+)')
    find_ner = re.finditer(pattern, text)
    for item in reversed(list(find_ner)):
        start_index, end_index = item.span()
        text = insert_tags(text, start_index, end_index)
    return text

assertEqual('ученый <>Хокинг</> из <>Великобритании</>', 
           tag_text('ученый Хокинг из Великобритании'))

In [22]:
marked_up_text = tag_text(text)

NameError: name 'text' is not defined

## Make draft files ##

In [34]:
for root, dirs, files in os.walk(test_texts_root):
    for file_name in files:
        input_path = test_texts_root + file_name
        output_path = output_root + file_name[:-3] + 'xml'
        text = tag_text(slurp(input_path))
        spit(output_path, text)

In [15]:
draft_texts_root = r'/home/nst/mount/data/share/yd/'\
               'popular_science_texts_store/ner_markup/test_ner/output_texts/'
    
final_texts_root = r'/home/nst/mount/data/share/yd/popular_science_texts_store/'\
                'ner_markup/final_markup/'

In [33]:
def change_tags(marked_texts_root):
    start_tag = '<>'
    final_tag = '</>'
    for root, dirs, files in os.walk(marked_texts_root):
        for file_name in files:
            file_path = draft_texts_root + file_name
            content = slurp(file_path)
            marked_text = re.sub(start_tag, '&', content)
            marked_text = re.sub(final_tag, '!&', marked_text)
            output_path = final_texts_root + file_name[:-3] + 'txt'
            spit(output_path, marked_text)
    print('I\'m done')

## Make final changes ##

In [25]:
change_tags(draft_texts_root)

I'm done
